## Overall

- Find ATAC-seq peaks that indicate chromatin accessibility (for episome similarity)
- 

### gkSVM


Enhanced Regulatory Sequence Prediction Using Gapped k-mer Features
Mahmoud Ghandi*, Dongwon Lee*, Morteza Mohammad-Noori, Michael A. Beer

Oligomers of length k, or k-mers, are convenient and widely used features for modeling the properties and functions of DNA and protein sequences. However, k-mers suffer from the inherent limitation that if the parameter k is increased to resolve longer features, the probability of observing any specific k-mer becomes very small, and k-mer counts approach a binary variable, with most k-mers absent and a few present once. Thus, any statistical learning approach using k-mers as features becomes susceptible to noisy training set k-mer frequencies once k becomes large. To address this problem, we introduce alternative feature sets using gapped k-mers, a new classifier, gkm-SVM, and a general method for robust estimation of k-mer frequencies. To make the method applicable to large-scale genome wide applications, we develop an efficient tree data structure for computing the kernel matrix. We show that compared to our original kmer-SVM and alternative approaches, our gkm-SVM predicts functional genomic regulatory elements and tissue specific enhancers with significantly improved accuracy, increasing the precision by up to a factor of two. We then show that gkm-SVM consistently outperforms kmer-SVM on human ENCODE ChIP-seq datasets, and further demonstrate the general utility of our method using a Naive-Bayes classifier. Although developed for regulatory sequence analysis, these methods can be applied to any sequence classification problem.

- !(GkmExplain)[https://watermark.silverchair.com/btz322.pdf?token=AQECAHi208BE49Ooan9kkhW_Ercy7Dm3ZL_9Cf3qfKAc485ysgAAArwwggK4BgkqhkiG9w0BBwagggKpMIICpQIBADCCAp4GCSqGSIb3DQEHATAeBglghkgBZQMEAS4wEQQMqLxQrjOLx-P7mGHnAgEQgIICbx9-MGXNlRedsyx7_ZL9GJKxe50xZkwXMReHYdSrhiDunJ8zo_fhsNZtuwIkz5vDK8JNTPJxTjOAFtgTDWJD_IymNT3aV00p3LBEKI1gC6qbPdGlBHgWnIXG1dMy1W3B7BqAO3KVZKwnKseVPR-6fk8JUYAAEkaZ9uKTFxYp8AnGMv9kLOUMZ1_F7CNXLW43xQt80TgazjjsZEag2svCIwIOK3-Hus8kyji2UlEcqTy-dsLnGsAsZLDyTLRbDNiaYv1QJXZP6awTZyK5jIIh4sPFzjajEwOKVs4qag03Peb6b45HEJqzymgOzk9qvZfK3GReDTNkiHgYMVlTiX7beS9TG98kbLjgTxBYYeKxBVGeYj3Hf6q0GBpRRpJnFsmVTCvKJ94rO-5MQq-99KquxkrlhLT1Y2JI-hsRNOG5ZMix1vcJWTe27XF7w521nvUBVTSkMm7Gn8rHJEIS3MA8izl9nI0yD7r2dHQmacAFlt-XWLfjNzMZKJU1Ydbwc5vm1Xetgin-Pyg0xw6oOdz3DcuQL07WiA3rzwce3JqQq48F2VWKuYtD8Msi2013TsNe_QJE1cf8AwCPV2ucIFyLuwXWGu7n88nlmrCftOW-CHz1pubQizgCKJ9qnMmDVZXHx_zTxC81k59xyZ8ldT15bKpZLRUkZw66oOo2whuIlvrTL1p9Qjk5joZoMijFbFf4R6wGH3w0QtMfTK-jCHg8Zn-oVSBnHRdJ_pVqi_LhI712QHlnmbf_ezuXVKF7TSDBatnDc2Fx1GYkB2mCovUKT5rOYFfR0XuvgA0b2TIKSIxXfNgIUxVSDi39Zg4qWW1n]:
- A full k-mer refers to a letter subsequence of length k—for example,
AAGT is a full 4-mer. By contrast, a gapped k-mer refers to a subsequence containing k letters and some number of gaps—for example,
A*AG*T is a gapped 4-mer containing 2 gaps (* is used to denote a
gap). In the gkm-SVM implementation, the parameter l denotes the
full length of the subsequences considered (including gaps), while k

### Deep Silencer


- "most of the silencers were collected from both the gkmSVM-based model and our DeepSilencer model. However, there are some unique silencers from DeepSilencer model. For example, chr3: 48,997,302-48,997,398 is a silencer predicted by DeepSilencer model, but not a silencer predicted by the gkmSVM-based model. And chr3: 48,997,266-48,997,466 that is a silencer validated in the human K562 cell line covers this predicted silencer."

In [ ]:
# from pytorch_lightning.loggers import WandbLogger  # newline 1
# from pytorch_lightning import Trainer

# wandb_logger = WandbLogger()  # newline 2
# trainer = Trainer(logger=wandb_logger)

In [ ]:
import os

import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader, random_split
import pytorch_lightning as pl
from pytorch_lightning.metrics.functional import accuracy


In [ ]:
class DNADataset(Dataset):
    """DNA + Pytorch Dataset"""
    def __init__(self, X, y, encoding='one-hot'):
        """
        Args:
            data (np.array): Path to the url .npz file with annotations.
        """
        if encoding==None:
            self.X = X
        else:
            self.X = torch.LongTensor(self.embedding_encoding(X))
        self.y = torch.FloatTensor(y) #binary labels
            #NOTE: we need this to be a float to accommodate nn.BCEWithLogitsLoss() later to match logit input.

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        sample = {'X': self.X[idx], 'y': self.y[idx]}
        return sample

    def embedding_encoding(self, seq_array: np.ndarray) -> np.ndarray:
        """
        :param seq_array: np array of DNA sequences
        :return: np array of integer encodings of input DNA sequences
        """
        nuc2id = {'A' : 0, 'C' : 1, 'T' : 2, 'G' : 3}
        N = len(seq_array)
        M = len(seq_array[0]) #since we know each sequence is the same length

        int_array = np.full((N,M),0)
        print('Nucleotides -> Integers')
        for seq_num, seq in enumerate(seq_array):
            for seq_idx, nucleotide in enumerate(seq):
                nuc_idx = nuc2id[nucleotide]
                int_array[seq_num, seq_idx] = nuc2id[nucleotide]
        return int_array


class DNADataModule(pl.LightningDataModule):
    def __init__(self, data_dir: str, batch_size:int=64,encoding='one-hot'):
        super().__init__()
        self.batch_size = batch_size
        self.data_fpath = data_dir
        self.encoding=encoding

    def setup(self, stage=None):
        all_data = np.load(self.data_fpath)

        self.train = DNADataset(all_data['train_seq'],all_data['train_y'],self.encoding)
        self.val   = DNADataset(all_data['train_seq'],all_data['train_y'],self.encoding)
        self.test  = DNADataset(all_data['test_seq'], all_data['test_y'], self.encoding)

    def train_dataloader(self):
        return DataLoader(self.train, batch_size=self.batch_size)

    def val_dataloader(self):
        return DataLoader(self.val, batch_size=self.batch_size)

    def test_dataloader(self):
        return DataLoader(self.test, batch_size=self.batch_size)

Loading Keras Weights (from DeepSilencer repo) into pytorch:
- caveats: keras stores as `double`, pytorch as `float`.


In [ ]:
class LitModel(pl.LightningModule):

    def __init__(self):
        super().__init__()
        self.encoder = nn.Sequential(nn.Linear(28 * 28, 128), nn.ReLU(), nn.Linear(128, 3))
        self.decoder = nn.Sequential(nn.Linear(3, 128), nn.ReLU(), nn.Linear(128, 28 * 28))

    def training_step(self, batch, batch_idx):
        # --------------------------
        # REPLACE WITH YOUR OWN
        x, y = batch
        x = x.view(x.size(0), -1)
        z = self.encoder(x)
        x_hat = self.decoder(z)
        loss = F.mse_loss(x_hat, x)
        self.log('train_loss', loss)
        return loss
        # --------------------------

    def validation_step(self, batch, batch_idx):
        # --------------------------
        # REPLACE WITH YOUR OWN
        x, y = batch
        x = x.view(x.size(0), -1)
        z = self.encoder(x)
        x_hat = self.decoder(z)
        loss = F.mse_loss(x_hat, x)
        self.log('val_loss', loss)
        # --------------------------

    def test_step(self, batch, batch_idx):
        # --------------------------
        # REPLACE WITH YOUR OWN
        x, y = batch
        x = x.view(x.size(0), -1)
        z = self.encoder(x)
        x_hat = self.decoder(z)
        loss = F.mse_loss(x_hat, x)
        self.log('test_loss', loss)
        # --------------------------

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        return optimizer